- 迁移学习：把在 source domain 任务上的学习到的模型应用到 target domain 的任务。
- Finetune 就是一种迁移学习的方法。比如做人脸识别，可以把 ImageNet 看作 source domain，人脸数据集看作 target domain。通常来说 source domain 要比 target domain 大得多。可以利用 ImageNet 训练好的网络应用到人脸识别中。
- 对于一个模型，通常可以分为前面的 feature extractor (卷积层)和后面的 classifier，在 Finetune 时，通常不改变 feature extractor 的权值，也就是冻结卷积层；并且改变最后一个全连接层的输出来适应目标任务，训练后面 classifier 的权值，这就是 Finetune。通常 target domain 的数据比较小，不足以训练全部参数，容易导致过拟合，因此不改变 feature extractor 的权值。
- Finetune 步骤如下：
    - 获取预训练模型的参数
    - 使用load_state_dict()把参数加载到模型中
    - 修改输出层
    - 固定 feature extractor 的参数。这部分通常有 2 种做法：
        - 固定卷积层的预训练参数。可以设置requires_grad=False或者lr=0
        - 可以通过params_group给 feature extractor 设置一个较小的学习率

### 不使用finetune
从零开始训练模型，这时只需要修改全连接层

In [ ]:
# 首先拿到 fc 层的输入个数
num_ftrs = resnet18_ft.fc.in_features
# 然后构造新的 fc 层替换原来的 fc 层
resnet18_ft.fc = nn.Linear(num_ftrs, classes)

### 使用finetune

In [ ]:
path_pretrained_model = enviroments.resnet18_path
state_dict_load = torch.load(path_pretrained_model)
resnet18_ft.load_state_dict(state_dict_load)

- 不冻结卷积层，所有层都是用相同的学习率  

- 冻结卷积层
    - 设置requires_grad=False
    - 这里先冻结所有参数，然后再替换全连接层，相当于冻结了卷积层的参数

In [ ]:
for param in resnet18_ft.parameters():
    param.requires_grad = False
    # 首先拿到 fc 层的输入个数
num_ftrs = resnet18_ft.fc.in_features
# 然后构造新的 fc 层替换原来的 fc 层
resnet18_ft.fc = nn.Linear(num_ftrs, classes)

- 设置学习率为 0
- 这里把卷积层的学习率设置为 0，需要在优化器里设置不同的学习率。首先获取全连接层参数的地址，然后使用 filter 过滤不属于全连接层的参数，也就是保留卷积层的参数；接着设置优化器的分组学习率，传入一个 list，包含 2 个元素，每个元素是字典，对应 2 个参数组。其中卷积层的学习率设置为 全连接层的 0.1 倍。

In [ ]:
# 首先获取全连接层参数的地址
fc_params_id = list(map(id, resnet18_ft.fc.parameters()))     # 返回的是parameters的 内存地址
# 然后使用 filter 过滤不属于全连接层的参数，也就是保留卷积层的参数
base_params = filter(lambda p: id(p) not in fc_params_id, resnet18_ft.parameters())
# 设置优化器的分组学习率，传入一个 list，包含 2 个元素，每个元素是字典，对应 2 个参数组
optimizer = optim.SGD([{'params': base_params, 'lr': 0}, 
                       {'params': resnet18_ft.fc.parameters(), 'lr': LR}], 
                       momentum=0.9)

- 使用分组学习率
- 这里不冻结卷积层，而是对卷积层使用较小的学习率，对全连接层使用较大的学习率，需要在优化器里设置不同的学习率。首先获取全连接层参数的地址，然后使用 filter 过滤不属于全连接层的参数，也就是保留卷积层的参数；接着设置优化器的分组学习率，传入一个 list，包含 2 个元素，每个元素是字典，对应 2 个参数组。其中卷积层的学习率设置为 全连接层的 0.1 倍。

- 使用 GPU 的 tips
    - PyTorch 模型使用 GPU，可以分为 3 步：
        1. 首先获取 device：device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        2. 把模型加载到 device：model.to(device)
        3. 在 data_loader 取数据的循环中，把每个 mini-batch 的数据和 label 加载到 device：inputs, labels = inputs.to(device), labels.to(device)